# Authenticate Twitter account

In [1]:
import tweepy

In [2]:
from keys import *

auth = tweepy.OAuthHandler(API_KEY, API_SECRET_KEY)
auth.set_access_token(TOKEN, TOKEN_SECRET)

api = tweepy.API(auth)

try:
    api.verify_credentials()
    print("Authentication OK")
except:
    print("Authentication Error")

Authentication OK


# Scrape eBird webpage

In [4]:
import urllib.request
import requests
import time
from bs4 import BeautifulSoup

### Verify a checklist exists

In [19]:
def check_checklist_exists(sub_id):
    '''
    Check that an eBird checklist exists
    
    Args:
        sub_id (int): submission id
    
    Returns:
        url if checklist exists
        False if checklist doesn't exist
    '''
    
    sub_id = 'S' + sub_id
    checklist_start = 'https://ebird.org/view/checklist/'
    url = checklist_start + sub_id
    
    response = requests.get(url)
    
    try:
        assert str(response) == '<Response [200]>'
        return response
    except:
        assert str(response) == '<Response [400]>'
        return False

In [6]:
import random

In [20]:
numbers = [str(random.randint(0, 59344389)) for _ in range(20000)]
responses = []
for number in numbers[:10]:
    time.sleep(1) # Wait between requests so not overloading eBird
    response = check_checklist_exists(number)
    if response: responses.append(response)
        
responses

[<Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>]

### Parse existing checklist

In [15]:
def get_soup(response):
    return BeautifulSoup(response.text, "html.parser")

In [9]:
def add_to_region(region, to_add, sep = ', '):
    '''
    Append to a string with a separator if necessary
    
    Inputs:
        region: the string to append to
        to_add: the thing to append to the string
        sep: the separator to add, if string
            already contains text
    
    Returns:
        the string with added text
    '''
    
    # If `region` already contains something
    if bool(region):
        region += sep + to_add
        return region
    else:
        region += to_add
        return region

In [10]:
def get_region(soup):
    
    region = ''
    
    # Find the region as linked to in the eBird text
    
    for link in soup.findAll('a'):
        # Find whether region is in the link address
        try:
            region_in = "region" in link['href']
        except:
            # No href tag
            continue

        # If region is in the link address, add it to the region string
        if region_in:
            string = link.string
            # For links in format <a href=''>string</a>
            if(bool(string)):
                region = add_to_region(region, string)

            # For links in format <a href=''><span>string</span></a>
            else:
                string = link.find('span').string
                region = add_to_region(region, string)

    return region

In [25]:
for response in responses:
    soup = get_soup(response)
    print(get_region(soup))

Cuyahoga County, Ohio, United States
Arica y Parinacota, Chile
Avalon Peninsula-St. John's County, Newfoundland and Labrador, Canada
Argenteuil County, Quebec, Canada
Rockingham County, New Hampshire, United States
Sonoma County, California, United States
Jefferson County, Colorado, United States
Wake County, North Carolina, United States
Minas Gerais, Brazil
